In [5]:
import pandas as pd
import numpy as np
import json
from scipy import stats
import tqdm

# Funzioni

In [26]:
def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

def nodeperc_withininterval(opinions, center, width):
    sortedops = sorted(opinions)
    sortedopsarr = np.array(sortedops)
    withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
    withinrange = withinrange[0]
    perc = len(withinrange) / len(sortedops)
    return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

def median_opinions(opinions):
    a = np.array(opinions)
    return np.median(a)
maxentr10 = entropy(np.random.uniform(0.0, 1.0, 250), 250, 11)

In [38]:
model = "triangles rewiring"

In [39]:
finalfile = f"{model} results.csv"

In [40]:
#create one aggregate file
import os
aggrfile = open(f"aggregate/{finalfile}", "w+")
header = "model,graph,n,p,pr,eps,gam,max_it\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open(f"aggregate/{finalfile}", "a+")
for graph in ['er', 'ba']:
    if graph == "er":
        p = 0.1
    else:
        p=5
    for pr in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for eps in [0.2, 0.3, 0.4]:
            for gam in [0.0, 0.4, 0.8, 1.2, 1.6]:
                name = f"{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000"
                s = f"{model},{graph},250,{p},{pr},{eps},{gam},100000\n"
                aggrfile.write(s)
aggrfile.close()

In [41]:
results = pd.read_csv(f"aggregate/{finalfile}")
results

,model,graph,n,p,pr,eps,gam,max_it
0,triangles rewiring,er,250,0.1,0.0,0.2,0.0,100000
1,triangles rewiring,er,250,0.1,0.0,0.2,0.4,100000
2,triangles rewiring,er,250,0.1,0.0,0.2,0.8,100000
3,triangles rewiring,er,250,0.1,0.0,0.2,1.2,100000
4,triangles rewiring,er,250,0.1,0.0,0.2,1.6,100000
...,...,...,...,...,...,...,...,...
175,triangles rewiring,ba,250,5.0,0.5,0.4,0.0,100000
176,triangles rewiring,ba,250,5.0,0.5,0.4,0.4,100000
177,triangles rewiring,ba,250,5.0,0.5,0.4,0.8,100000
178,triangles rewiring,ba,250,5.0,0.5,0.4,1.2,100000


In [42]:
avg = []
std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_iterations {name}.json')
    data = json.load(jsonfile)
    finalitarr = []
    for nr in data.keys():
        finalitarr.append(data[nr])
    finalitarr = np.array(finalitarr)
    avg.append(np.average(finalitarr))
    std.append(np.std(finalitarr))
results["avg_nit"] = avg
results["std_nit"] = std
results["nruns"] = int(nr)+1

doing triangles rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.8 m

In [43]:
avg01 = []
std01 = []
avg001=[]
std001=[]
avg00001=[]
std00001=[]
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray01 = []
    ncarray001 = []
    ncarray00001 = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc01 = nclusters(finalops, 0.1)
        nc001 = nclusters(finalops, 0.01)
        nc00001 = nclusters(finalops, 0.0001)
        ncarray01.append(nc01)
        ncarray001.append(nc001)
        ncarray00001.append(nc00001)
    ncarray01 = np.array(ncarray01)
    ncarray001 = np.array(ncarray001)
    ncarray00001 = np.array(ncarray00001)
    avg01.append(np.average(ncarray01))
    std01.append(np.std(ncarray01))
    avg001.append(np.average(ncarray001))
    std001.append(np.std(ncarray001))
    avg00001.append(np.average(ncarray00001))
    std00001.append(np.std(ncarray00001))
results["avg_ncluster_01"] = avg01
results["std_ncluster_01"] = std01
results["avg_ncluster_001"] = avg001
results["std_ncluster_001"] = std001
results["avg_ncluster_00001"] = avg00001
results["std_ncluster_00001"] = std00001

doing triangles rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.8 m

In [44]:
results.to_csv(f"aggregate/{finalfile}")

In [45]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001
0,triangles rewiring,er,250,0.1,0.0,0.2,0.0,100000,16.600000,0.916515,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
1,triangles rewiring,er,250,0.1,0.0,0.2,0.4,100000,31.633333,16.055079,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
2,triangles rewiring,er,250,0.1,0.0,0.2,0.8,100000,9919.833333,10583.807815,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
3,triangles rewiring,er,250,0.1,0.0,0.2,1.2,100000,85785.600000,28390.598434,15,1.616291,0.443752,1.616291,0.443752,1.616291,0.443752
4,triangles rewiring,er,250,0.1,0.0,0.2,1.6,100000,81342.366667,34956.042465,15,1.641162,0.404782,1.641162,0.404782,1.641162,0.404782


In [46]:
averageops = []
averagestdops = []
averagemedianops = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    avgarray = []
    stdarray = []
    medarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        avg = average_opinions(finalops)
        med = median_opinions(finalops)
        std = std_opinions(finalops)
        avgarray.append(avg)
        medarray.append(med)
        stdarray.append(std)
    avgarray = np.array(avgarray)
    medarray = np.array(medarray)
    stdarray = np.array(stdarray)
    averageops.append(np.average(avgarray))
    averagestdops.append(np.average(stdarray))
    averagemedianops.append(np.average(medarray))
results["avg_mean_opinion"] = averageops
results["avg_std_opinions"] = averagestdops
results["avg_median_opinion"] = averagemedianops

doing triangles rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.8 m

In [47]:
results.to_csv(f"aggregate/{finalfile}")

In [48]:
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,nruns,avg_ncluster_01,std_ncluster_01,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion
0,triangles rewiring,er,250,0.1,0.0,0.2,0.0,100000,16.600000,0.916515,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.488209,1.360222e-06,0.488209
1,triangles rewiring,er,250,0.1,0.0,0.2,0.4,100000,31.633333,16.055079,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.444246,6.701247e-07,0.444246
2,triangles rewiring,er,250,0.1,0.0,0.2,0.8,100000,9919.833333,10583.807815,15,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.536992,7.123931e-17,0.536992
3,triangles rewiring,er,250,0.1,0.0,0.2,1.2,100000,85785.600000,28390.598434,15,1.616291,0.443752,1.616291,0.443752,1.616291,0.443752,0.512175,1.712722e-01,0.513810
4,triangles rewiring,er,250,0.1,0.0,0.2,1.6,100000,81342.366667,34956.042465,15,1.641162,0.404782,1.641162,0.404782,1.641162,0.404782,0.473251,1.690076e-01,0.487112


In [49]:
values_avg = []
values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'{model} {graph}{p} pr{pr} e{eps} g{gam} mi100000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    entrarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        entr = entropy(finalops, 100, 11)
        entrarray.append(entr)
    entrarray = np.array(entrarray)
    values_avg.append(np.average(entrarray))
    values_std.append(np.std(entrarray))

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std

results['10B_avg_norm_entr'] = results['10B_avg_entr'].apply(lambda x: x/maxentr10)

doing triangles rewiring er0.1 pr0.0 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.6 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.0 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.4 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g0.8 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.2 mi100000
doing triangles rewiring er0.1 pr0.0 e0.4 g1.6 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.0 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.4 mi100000
doing triangles rewiring er0.1 pr0.1 e0.2 g0.8 m

In [50]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,triangles rewiring,er,250,0.1,0.0,0.2,0.0,100000,16.600000,0.916515,...,1.000000,0.000000,1.000000,0.000000,0.488209,1.360222e-06,0.488209,0.000000,0.000000,0.000000
1,triangles rewiring,er,250,0.1,0.0,0.2,0.4,100000,31.633333,16.055079,...,1.000000,0.000000,1.000000,0.000000,0.444246,6.701247e-07,0.444246,0.000000,0.000000,0.000000
2,triangles rewiring,er,250,0.1,0.0,0.2,0.8,100000,9919.833333,10583.807815,...,1.000000,0.000000,1.000000,0.000000,0.536992,7.123931e-17,0.536992,0.000000,0.000000,0.000000
3,triangles rewiring,er,250,0.1,0.0,0.2,1.2,100000,85785.600000,28390.598434,...,1.616291,0.443752,1.616291,0.443752,0.512175,1.712722e-01,0.513810,0.451187,0.310671,0.197541
4,triangles rewiring,er,250,0.1,0.0,0.2,1.6,100000,81342.366667,34956.042465,...,1.641162,0.404782,1.641162,0.404782,0.473251,1.690076e-01,0.487112,0.480489,0.292323,0.210370


In [51]:
results.to_csv("aggregate/{}".format(finalfile))
results = pd.read_csv("aggregate/{}".format(finalfile), index_col=[0])

In [52]:
results.head()

,model,graph,n,p,pr,eps,gam,max_it,avg_nit,std_nit,...,avg_ncluster_001,std_ncluster_001,avg_ncluster_00001,std_ncluster_00001,avg_mean_opinion,avg_std_opinions,avg_median_opinion,10B_avg_entr,10B_std_entr,10B_avg_norm_entr
0,triangles rewiring,er,250,0.1,0.0,0.2,0.0,100000,16.600000,0.916515,...,1.000000,0.000000,1.000000,0.000000,0.488209,1.360222e-06,0.488209,0.000000,0.000000,0.000000
1,triangles rewiring,er,250,0.1,0.0,0.2,0.4,100000,31.633333,16.055079,...,1.000000,0.000000,1.000000,0.000000,0.444246,6.701247e-07,0.444246,0.000000,0.000000,0.000000
2,triangles rewiring,er,250,0.1,0.0,0.2,0.8,100000,9919.833333,10583.807815,...,1.000000,0.000000,1.000000,0.000000,0.536992,7.123931e-17,0.536992,0.000000,0.000000,0.000000
3,triangles rewiring,er,250,0.1,0.0,0.2,1.2,100000,85785.600000,28390.598434,...,1.616291,0.443752,1.616291,0.443752,0.512175,1.712722e-01,0.513810,0.451187,0.310671,0.197541
4,triangles rewiring,er,250,0.1,0.0,0.2,1.6,100000,81342.366667,34956.042465,...,1.641162,0.404782,1.641162,0.404782,0.473251,1.690076e-01,0.487112,0.480489,0.292323,0.210370


Unused functions

# def nclusters_meanshift(data, bw):
#     ops = {i: data[i] for i in range(len(data))}
#     sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
#     A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
#     clustering = MeanShift(bandwidth=bw).fit(A)
#     lbls = clustering.labels_
#     labels = np.arange(len(sorted_ops))
#     for i in range(len(labels)):
#         cl = lbls[i]
#         labels[sorted_ops[i][0]]=cl
#     cluster_participation_dict = {}
#     for l in labels:
#         if l not in cluster_participation_dict:
#             cluster_participation_dict[l] = 1
#         else:
#             cluster_participation_dict[l] += 1
#     #computing effective number of clusters using function explained in the paper
#     C_num = 0
#     C_den = 0
#     for k in cluster_participation_dict:
#         C_num += cluster_participation_dict[k]
#         C_den += ((cluster_participation_dict[k])**2)
#     C_num = (C_num**2)
#     C = C_num/C_den
#     return C

In [ ]:
def steady_state_coevolving(model, name, max_iterations=100000, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=False):
    if not os.path.exists(f"snapshotGraphs/edgelist {name} 0.csv"):
        print(type(model.graph.graph))
        system_status = []
        steady_it = 0
        for it in tqdm.tqdm(range(0, max_iterations), disable=not progress_bar):            
            its = model.iteration(node_status)
            
            if it == 0:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    first_values = list(its['status'].values())
                    for op in first_values:
                        opfile.write(str(op)+"\n")

            elif it > 0:
                old = np.array(list(system_status[-1]['status'].values()))
                actual = np.array(list(its['status'].values()))
                # res = np.abs(old - actual)
                if np.all((its['max_diff'] < sensibility)):
                    steady_it += 1
                else:
                    steady_it = 0

                if it % 500 == 0:
                    G = model.graph.graph
                    nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                    with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                        for op in list(actual):
                            opfile.write(str(op)+"\n")

            system_status.append(its)
            
            if steady_it == nsteady:
                G = model.graph.graph
                nx.write_edgelist(G, f"snapshotGraphs/edgelist {name} {it}.csv", delimiter=",")
                with open(f"snapshotGraphs/opinions {name} {it}.txt", "w") as opfile:
                    for op in list(actual):
                        opfile.write(str(op)+"\n")
                        
                return system_status[:-nsteady]

        return system_status
# Save snapshots of graph during evolution
import sys
sys.path.append("/home/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/")
sys.path.append("/data1/users/pansanella/mydata/GitHub/local_packages/netdspatch_local/")

import json
import networkx as nx
import ndlib_local.ndlib.models.ModelConfig as mc
import ndlib_local.ndlib.models.opinions as op
import warnings
warnings.filterwarnings("ignore")
n = 250
max_it = 100000
for graphname in ['er', 'ba']:
    if graphname == 'er':
        p = 0.1
        graph = nx.erdos_renyi_graph(n, p)
    else:
        p = 5
        graph = nx.barabasi_albert_graph(n, p)
    for pr in [0.5]:
        for e in [0.2, 0.3, 0.4]:
            for g in [0.0, 0.5, 1.0, 1.5]:
                name = f"rewiring {graphname}{p} pr{pr} e{e} g{g} mi{max_it}"
                model = op.AdaptiveAlgorithmicBiasModel(graph)
                config = mc.Configuration()
                config.add_model_parameter("epsilon", e)
                config.add_model_parameter("gamma", g)
                config.add_model_parameter("p", pr)
                model.set_initial_status(config)
                steady_status = steady_state_coevolving(model=model, name=name, max_iterations=max_it, nsteady=1000, sensibility=0.00001, node_status=True, progress_bar=True)                                